In [1]:
import sys
import os
import time

import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow import keras

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive

    drive.mount('/gdrive')
    % cd / gdrive / My Drive / ANNDL-H1 /
    !pip install sklearn
    !pip install visualkeras

import visualkeras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
import keras_cv as keras_cv
import tensorflow_datasets as tfds

#OUR CUSTOM HELPER CLASS
sys.path.append('../../../Utils/')
from DatasetHelper import *
from ModelHelper import ModelHelper

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.10.0


In [2]:

dataset_dir = '../../data'
training_dir = os.path.join(dataset_dir, '')
#Create Instances Of Utils Helper
seed = 5000
labels = {0: 'Species1', 1: 'Species2', 2: 'Species3', 3: 'Species4', 4: 'Species5', 5: 'Species6', 6: 'Species7',
          7: 'Species8'}

#Dataset Helper
dataset_helper = DatasetHelper("../../", seed, create_dirs=True)  #Take in input dataset folder and random seed

#Model Helper
#SAVE MODEL IN LOCAL, IF MODEL IS GOOD; COPY IT BY HAND TO good_model Folder
model_helper = ModelHelper("../../", labels)  #take in input local models folder and lables
model_helper.create_seed(tf, seed);

#Load Dataset
X, Y = dataset_helper.load_dataset_from_numpy()
dataset_size = X.shape[0]
#Split Training and Testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=seed, test_size=int(0.1 * dataset_size),
                                                    stratify=Y)

# Normalize data
X_train = X_train / 255.
X_test = X_test / 255.

#Split Training and Validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=seed,
                                                  test_size=int(0.1 * dataset_size), stratify=Y_train)
old_y_train = Y_train

In [3]:
DatasetHelper.keras_cv_augmentation(X_train, Y_train, 0.05)

Metal device set to: Apple M1 Pro


2022-11-22 00:52:09.488310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-22 00:52:09.488438: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-22 00:52:09.931428: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-22 00:52:11.318901: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(array([[[[0.6431373 , 0.68235296, 0.6509804 ],
          [0.60784316, 0.63529414, 0.60784316],
          [0.46666667, 0.4627451 , 0.44313726],
          ...,
          [0.13333334, 0.24313726, 0.19607843],
          [0.13333334, 0.25882354, 0.20784314],
          [0.16862746, 0.30980393, 0.25490198]],
 
         [[0.87058824, 0.9098039 , 0.8784314 ],
          [0.6666667 , 0.69411767, 0.6666667 ],
          [0.43529412, 0.4392157 , 0.41568628],
          ...,
          [0.21176471, 0.3137255 , 0.27058825],
          [0.25490198, 0.38039216, 0.32941177],
          [0.14509805, 0.28627452, 0.23137255]],
 
         [[0.9254902 , 0.972549  , 0.93333334],
          [0.9137255 , 0.9411765 , 0.9098039 ],
          [0.7607843 , 0.7647059 , 0.7411765 ],
          ...,
          [0.3647059 , 0.46666667, 0.41568628],
          [0.4509804 , 0.5686275 , 0.52156866],
          [0.3647059 , 0.49803922, 0.44313726]],
 
         ...,
 
         [[0.7254902 , 0.7058824 , 0.68235296],
          [0.69019